<a href="https://colab.research.google.com/github/nyp-sit/iti107/blob/main/session-10/cnn_lstm_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Anomaly Detection 
                                                             
<center>
    <img src="https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/resources/video-anomaly.png" height="267" width="500" style="vertical-align:middle;margin:10px 20px"/>
</center>
                

In this lab,we will train a convolutional LSTM autoencoder model to learn what is normal pedestrian traffic and then use it to detect unusual activities. This technique is described in this [paper](https://arxiv.org/pdf/1701.01546.pdf)



## Import libraries


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import os
from IPython.display import display
from IPython.display import Image as ipyImage
from PIL import Image
import numpy as np
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

## Dataset 

**UCSD Anomaly Detection Dataset**

The UCSD Anomaly Detection Dataset is a set of video frames from a stationary camera overlooking pedestrian walkways. The crowd density in the walkways was variable, ranging from sparse to very crowded. In the normal setting, the video contains only pedestrians. Abnormal events include bikers, skaters, small carts, and people walking across a walkway or in the grass that surrounds it. We will only use the *Peds1* subset, which contains 34 training video samples (normal scenes) and 36 testing video samples (anomalous scenes).


### Download the Dataset


In [ ]:
dataset_url  = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/UCSDped1.zip'
path_to_zip = keras.utils.get_file('UCSDped1.zip', origin=dataset_url, extract=True, cache_dir='.')
dataset_dir = os.path.join(os.path.dirname(path_to_zip), 'UCSDped1')

After unzipping the file, you will see two subfolders: `Train` and `Test`. 
The training data (consists of 24 video clips) are in Train subfolder. Each clip is in a separate subfolder 'Train001', 'Train002', etc, and each of these subfolders contains 200 image frames. Similarly for the test data.

In the code below, we are just setting up all the filepaths to be used later.

In [ ]:
# setup all the relative path
train_dir = os.path.join(dataset_dir, 'Train')
test_dir = os.path.join(dataset_dir, 'Test')

### Visualize the Train dataset

Our training set contains only video scenes that are 'normal'. We will convert the image frames to a 'video' (actually as animated gif) for easier viewing. The video consists of 200 frames. From the left navigation panel, you will see that a gif file called <train_sample_folder_name>.gif has been created, e.g. Train034.gif.

Let's just define a util function to create gif from series of images.

In [ ]:
def create_gif(image_folder, output_file, img_type='png'):
    # Create the frames
    frames = []

    # files need to be sorted from 1...n so that the video is played in correct sequence
    imgs = sorted(glob.glob(f'{image_folder}/*.{img_type}'))
    for i in imgs:
        new_frame = Image.open(i)
        frames.append(new_frame)
    
    # Save into a GIF file that loops forever
    frames[0].save(output_file, format='gif',
                append_images=frames[1:],
                save_all=True,
                duration=120, loop=0)


In [ ]:
# You can change the following train_sample_folder to another folder to view other clips
train_sample_folder = 'Train034' 
image_folder = os.path.join(train_dir, train_sample_folder)
gif_filename = train_sample_folder + '.gif' 
create_gif(image_folder, gif_filename, img_type='tif')
with open(gif_filename,'rb') as file:
    display(ipyImage(file.read(), format='png'))

## Visualize the Test dataset

Let us visualize the video frames from the test dataset folder Test024, as an animated gif. You should be able to see some anomalous event (e.g. a small cart driven on the walkway) in the animated gif you create. 

In [ ]:
# set the test sample folder to folder of Test001 and set the image_folder accordingly
test_sample_folder = 'Test024' 
image_folder = os.path.join(test_dir, test_sample_folder) 

create_gif(image_folder, gif_filename, img_type='tif')

with open(gif_filename,'rb') as file:
    display(ipyImage(file.read(), format='png'))

### Prepare Training Dataset

As we will be using a convolutional LSTM to learn to reconstruct the video frames, we will prepare our data as follows: 

1. Divide the training video frames into temporal sequences, each of size 10 using the sliding window technique.
2. Resize each frame to `IMG_HEIGHT × IMG_WIDTH` to ensure that input images have the same resolution.
3. Scale the pixels values between 0 and 1 by dividing each pixel by 255.

To increase the number of samples available, data augmentation in the temporal dimension is done by concatenating
frames with stride-1, stride-2, and stride-3, as suggested in the [paper](https://arxiv.org/pdf/1701.01546.pdf). For example, the first stride-1 sequence is made up of frames `{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}`, whereas the first
stride-2 sequence contains frames `{1, 3, 5, 7, 9, 11, 13, 15, 17, 19}`, and
stride-3 sequence would contains frames `{1, 4, 7, 10, 13, 16, 19, 22, 25,
28}`


In [ ]:
def augment_clips(stride, frames_list, sequence_length):
   
    augmented_clips = []
    sz = len(frames_list)
    img_height, img_width = frames_list[0].shape[0], frames_list[0].shape[1]
    clip = np.zeros(shape=(sequence_length, img_height, img_width, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_length:
                augmented_clips.append(np.copy(clip))
                cnt = 0
    return augmented_clips


def prepare_dataset(fileset, img_height, img_width, strides=1, batch_size=1, sequence_length=10):
    
    clips = []
    all_frames = []
    # important to sort the files as the video sequence must be in the correct order
    filepaths = sorted(glob.glob(fileset))
    
    for fpath in filepaths:
        img = Image.open(fpath).resize((img_height, img_width))
        img = np.array(img, dtype=np.float32) / 255.0
        all_frames.append(img)
    
    for stride in range(1, 1+strides):
        clips.extend(augment_clips(stride=stride, 
                                   frames_list=all_frames,
                                   sequence_length=sequence_length))
    

    # convert the list to 4D numpy array
    clips = np.stack(clips, axis=0)

    # Y = X for autoencoder
    dataset = tf.data.Dataset.from_tensor_slices( (clips, clips) ).batch(batch_size)

    return dataset
    


In [ ]:
IMG_HEIGHT=128
IMG_WIDTH=128
SEQUENCE_LENGTH=10
BATCH_SIZE=4

train_fileset = os.path.join(train_dir, '*/*.tif')
train_dataset = prepare_dataset(train_fileset,
                                IMG_HEIGHT, 
                                IMG_WIDTH, 
                                batch_size=BATCH_SIZE,
                                strides=2,
                                sequence_length=SEQUENCE_LENGTH)

Let's take a look at the shape of one of the sample. Note that this is a 5D tensor, where the axis 1 represents the time-steps, and axis 2 and axis 3 represents the image heigth and width.

In [ ]:
sample, _ = next(train_dataset.as_numpy_iterator())
print(sample.shape)

### Building the Autoencoder Model


![autoencoder](https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/iti107/resources/convlstm_autoencoder.png)


The spatial encoder (the Conv2D layers) takes in one
image frame at each timestep, and pass the output to the temporal encoder (the LSTM layers) for each timestep, for T (e.g., 10) timesteps. The decoders mirror the encoders to reconstruct the video
volume. To apply Conv2D for T timesteps, we use the TimeDistributed wrapper provided by Keras API.


In [ ]:
def get_model():

    inputs = keras.Input(shape=(SEQUENCE_LENGTH, IMG_HEIGHT, IMG_HEIGHT, 1))
    conv = keras.layers.Conv2D(128, (11,11), strides=4, padding='same')
    x = keras.layers.TimeDistributed(conv)(inputs)
    x = keras.layers.LayerNormalization()(x)
    conv = keras.layers.Conv2D(64, (5, 5), strides=2, padding="same")
    x = keras.layers.TimeDistributed(conv)(x)
    x = keras.layers.LayerNormalization()(x)

    x = keras.layers.ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True)(x)
    x = keras.layers.LayerNormalization()(x)

    conv = keras.layers.Conv2DTranspose(64, (5, 5), strides=2, padding="same")
    x = keras.layers.TimeDistributed(conv)(x)
    x = keras.layers.LayerNormalization()(x)

    conv = keras.layers.Conv2DTranspose(128, (11, 11), strides=4, padding="same")
    x = keras.layers.TimeDistributed(conv)(x)
    x = keras.layers.LayerNormalization()(x)

    conv = keras.layers.Conv2D(1, (11, 11), activation="sigmoid", padding="same")
    outputs = keras.layers.TimeDistributed(conv)(x)

    model = keras.Model(inputs=[inputs], outputs=[outputs])

    return model


In [ ]:
model = get_model()
model.summary()

As we aims to minimise the reconstruction error, one appropriate loss function to use is the Mean Squared Error (MSE).


In [ ]:
model.compile(loss=tf.keras.losses.MeanSquaredError(), 
        optimizer=tf.keras.optimizers.Adam(lr=1e-4, decay=1e-5, epsilon=1e-5),
        metrics=['mse'])


Let's the training begin!! This might take a while so *sit back, relax and wait!*

In [ ]:
num_epochs = 3

model.fit(train_dataset, epochs=3)


In [ ]:
model.save("anomaly_detector")

You can also download the model that we have trained if you cannot wait for the training to finish.  Just uncomment the following lines of codes:

In [ ]:
# !wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/pretrained-models/anomaly_detector.zip
# !unzip anomaly_detector.zip

### Prepare Testing Dataset

Now we will create a test dataset that we can use to test the trained auto-encoder. 


In [ ]:
def prepare_test_sequence(fileset, img_height, img_width, sequence_length): 

    filepaths = sorted(glob.glob(fileset))
    img_array = np.zeros((len(filepaths), img_height, img_width, 1)).astype(np.float32)

    for i, fpath in enumerate(filepaths):
        im = Image.open(fpath)
        im = im.resize((img_height, img_width))
        img_array[i, :, :, 0] = np.array(im)/255.0

    sz = img_array.shape[0] - sequence_length + 1
    sequences = np.zeros((sz, sequence_length, img_height, img_width, 1))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((sequence_length, img_height, img_width, 1))
        for j in range(0, sequence_length):
            clip[j] = img_array[i + j, :, :, :]
        sequences[i] = clip

    return sequences

In [ ]:
test_fileset = os.path.join(test_dir, 'Test024/*.tif')
test_sequence = prepare_test_sequence(test_fileset,
                                IMG_HEIGHT, 
                                IMG_WIDTH, 
                                sequence_length=SEQUENCE_LENGTH)

In [ ]:
print(test_sequence.shape)

#### Abnormality scores of video sequences

The following codes feeds the sequences of video frames from the test video to reconstruct the video sequence. We then compute the reconstruction losses and use that to compute the normalized abnormality scores of the video sequence.  

In [ ]:
model = keras.models.load_model('anomaly_detector')

In [ ]:
reconstructed_sequences = model.predict(test_sequence,batch_size=4)

In [ ]:
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(test_sequence[i],reconstructed_sequences[i])) for i in range(0, test_sequence.shape[0])])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)

In [ ]:
plt.plot(sa)
plt.ylabel('anomaly score Sa(t)')
plt.xlabel('frame t')
plt.show()

In [ ]:
tmp_folder = 'anom_plots'

def plot_anomaly_scores(original, reconstructed, scores, max_score, frame_num):

    if not os.path.exists(tmp_folder):
        os.mkdir(tmp_folder)
    plt.ioff()
    fig = plt.figure(figsize=(10, 4))
   
    x = np.arange(0, 200)
    y = np.zeros(200)

    # show original image
    fig.add_subplot(131)
    plt.title(f'frame {frame_num}')
    plt.set_cmap('gray')
    plt.imshow(original)

    fig.add_subplot(132)
    plt.title(f'frame {frame_num}')
    plt.set_cmap('gray')
    plt.imshow(reconstructed)

    fig.add_subplot(133)
 
    plt.ylim(0, max_score + 0.1)
    plt.title('anomaly scores')
    x1 = np.arange(0, 200)
    y1 = np.zeros(200)
    plt.plot(x1, y1)
    plt.plot(scores)

    fig.savefig(os.path.join(tmp_folder, '{:0>3d}.png'.format(frame_num)))

    plt.ioff()
    plt.close()
  
    
def create_video_animation(scores, orig_sequences, reconstructed_sequences,gif_file):

    anom_scores = []
    counter = 0
    num_frames = orig_sequences.shape[0]
    max_score = np.max(scores)

    for frame_num in tqdm(range(num_frames)):
        anom_scores.append(scores[frame_num])
        # display the last frame of each video sequence
        plot_anomaly_scores(orig_sequences[frame_num, 9, :, :, 0], 
                            reconstructed_sequences[frame_num, 9, :, :, 0],
                            anom_scores, 
                            max_score, 
                            frame_num)
     
    
    create_gif(tmp_folder, gif_file)

In [ ]:
create_video_animation(sa, test_sequence, reconstructed_sequences, "anomaly.gif")
with open('anomaly.gif','rb') as file:
    display(ipyImage(file.read(), format='png', embed=True))
